In [16]:
# import requests
# import pandas as pd
# from all_comps import climbing_category, competition_overview_df
import all_comps

# for comp in competition_overview_df:
base_url = 'http://egw.ifsc-climbing.org/egw/ranking/json.php?'
# competition_id = comp['compId']
competition_id = '7923'

if competition_overview_df[competition_overview_df['compId'] == competition_id]['isBoulderingComp'] == 1:    
    for category in ['5', '6']:
        df_category = climbing_category(category)

        for stage in ['0', '1', '2', '3']:
            df_stage = competition_stage(stage)

            url = base_url + 'comp=' + competition_id + '&cat=' + category + '&route=' + stage
            comp_json = requests.get(url).json()

            # Adding stage information to main competition overview dataframe
            competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'isoOpen' + df_stage + df_category] = comp_json.get('route_iso_open')
            competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'isoClose' + df_stage + df_category] = comp_json.get('route_iso_close')
            competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'stageStart' + df_stage + df_category] = comp_json.get('route_start')
            competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'numProblems' + df_stage + df_category] = comp_json.get('route_num_problems')
            competition_overview_df.loc[competition_overview_df['compId'] == competition_id, 'climbingTime' + df_stage + df_category] = comp_json.get('route_climbing_time')

# competition_overview_df
# comp_json


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
def get_stage_results(url_json):
    new_stage_rows = []    
    for climber in comp_json['participants']:
        new_comp_stage = dict()
        
        # This same information needs to be in the row for every climber
        new_comp_stage.update({
            'compId': comp_json.get('WetId'),
            'category': climbing_category(comp_json.get('GrpId')),
            'stage': competition_stage(comp_json['route_order'])
        })

        # Get climber's results for this stage
        new_comp_stage.update({
            'climberID': climber.get('PerId'),
            'startOrder': climber.get('start_order'),
            'rankAfterStage' : climber.get('result_rank'),
            'boulder1': climber.get('boulder1'),
            'boulder2': climber.get('boulder2'),
            'boulder3': climber.get('boulder3'),
            'boulder4': climber.get('boulder4'),
            'boulder5': climber.get('boulder5'),
        })
        
        # Append new row dictionary, (representing a stage of a comp), in the new_stage_rows list
        new_stage_rows.append(new_comp_stage)

    # Return new rows to insert in dataframe
    return new_stage_rows

In [ ]:
def competition_stage(stage, intext=1):
    external_stages = {
        '-1': 'General result',
        '0': '1. Qualification',
        '1': '2. Qualification',
        '2': 'Semi-Final',
        '3': 'Final'
    }
    df_stages = {
        '0': 'Q1',
        '1': 'Q2',
        '2': 'SF',
        '3': 'F'
    }
    return df_stages.get(stage, 'Other') if intext == 1 else external_stages.get(stage, 'Other')